In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
from scipy.constants import e, hbar
from scipy.linalg import solve_sylvester

from two_coupled_qubits import ChargeQubit, Resonator

In [3]:
q1 = ChargeQubit(e_c=0.23084, e_j=16.34, levels=1, n_g=0)
q2 = ChargeQubit(e_c=0.26084, e_j=16.34, levels=1, n_g=0)
r = Resonator(dim=5, w=7.50, c_r=368.440)

In [4]:
# Parameters
c_g = 1.946
beta_1 = c_g/(c_g + q1.c_s)
beta_2 = c_g/(c_g + q2.c_s)

# Operators
b = destroy(r.dim)

# Free Hamiltonian
h0 = tensor(q1.h, q2.h, r.h)

# Interacting Hamiltonian
h1r = tensor(- 1j * 2 * beta_1 * e * r.v_rms / hbar * 1e-9 * q1.number[1], qeye(q2.dim), (b - b.dag()))
h2r = tensor(qeye(q1.dim), - 1j * 2 * beta_2 * e * r.v_rms / hbar * 1e-9 * q2.number[1], (b - b.dag()))
hint = h1r + h2r

# Total Hamiltonian
h = h0 + hint

In [21]:
_, states_q1 = q1.h.eigenstates()
_, states_q2 = q2.h.eigenstates()

p = tensor(states_q1[0] * states_q1[0].dag() + states_q1[1] * states_q1[1].dag(), states_q2[0] * states_q2[0].dag() + states_q2[1] * states_q2[1].dag(), qeye(r.dim))

s = Qobj(solve_sylvester(-h0.full(), h0.full(), -potential.full()))

heff = Qobj(h0.full() + 0.5 * commutator(s.full(), potential.full())).full()

In [23]:
heffective = Qobj(p * heff * p)

In [24]:
eigenenergies, eigenstates = heffective.eigenstates()

In [25]:
# Eigenvectors
eigenenergies, eigenstates = heffective.eigenstates()

# States
psi = np.empty((2, 2), dtype=Qobj)

for i in range(2):
    for j in range(2):
            psi[i,j] = tensor(eigenstates[i], eigenstates[j])

# Time
t_list = np.linspace(0, 20, 1000)

In [26]:
heffective

Quantum object: dims = [[45], [45]], shape = (45, 45), type = oper, isherm = True
Qobj data =
[[  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.          35.20624372   0.         ...   0.           0.
    0.        ]
 [  0.           0.          70.41248745 ... 147.77858251   0.
    0.        ]
 ...
 [  0.           0.         147.77858251 ...  70.41248745   0.
    0.        ]
 [  0.           0.           0.         ...   0.         105.61873117
    0.        ]
 [  0.           0.           0.         ...   0.           0.
  140.82497489]]

In [27]:
# Master Equation
result = mesolve(H=heffective, rho0=psi[1,0].full(), tlist=t_list, c_ops=[], e_ops=[], args={})

AttributeError: 'numpy.ndarray' object has no attribute 'isoper'

In [11]:
# Expected value
prob = np.empty((q1.dim, q2.dim, r.dim), dtype=Qobj)

for i in range(q1.dim):
    for j in range(q2.dim):
        for k in range(r.dim):
            prob[i,j,k] = expect(psi[i,j,k]*psi[i,j,k].dag(), result.states)

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed